# 命令型モデル

- 命令型モデルでは、「計算」は「命令を実行する」ことと捉える
    - 命令: 計算機の状態を変更する処理
    - 命令を実行することで計算機の記憶装置や表示装置の中身が変化していき、最終的に得られた状態を計算の結果と考える

## チューリングマシン

- チューリングマシン: 命令型モデルの原型
    - 左右に無限の長さを持つ「テープ」に接する「ヘッド」を持つ
    - 「ヘッド」は「テーブル」と呼ばれる制御装置により左右に移動され、テープから情報を読み書きする
- チューリングマシンの処理:
    1. ヘッドが接するテープのマス目から情報を読み込む
    2. ヘッドが接するテープのマス目に情報を書き込む
    3. ヘッドをテープ上で左右に1つ移動する
    4. テーブルに格納されている状態を変更する
    5. テーブルに格納されている状態が最終状態に到達するまで 1 - 4 の処理を反復する

### 定義
- `TuringTape::Dict{Int, TuringValue}`: テープ
    - `TuringValue::Union{Val{0}, Val{1}, Nothing}`: テープに書き込み可能な値
        - `val(::Val{T}) where {T} = T`: TuringValue の値を取得する関数
    - 左右に無限の長さを持つ配列
    - テープの各マスには TuringValue (value: 0 or 1 or nothing) を書き込み可能
- `TuringCommand::struct`: コマンド構造体
    - Fields:
        - `write::TuringValue`: テープに書き込む値
        - `move::Union{Val{-1}, Val{1}}`: ヘッドを動かす方向
        - `next::Symbol`: 次の状態
- `TuringInstruction::struct`: 命令構造体
    - Fields:
        - `command::TuringCommand`: ヘッドが接するテープのマス目に値が設定されていない場合に実行されるコマンド
        - `command0::TuringCommand`: ヘッドが接するテープのマス目の値が 0 の場合に実行されるコマンド
        - `command1::TuringCommand`: ヘッドが接するテープのマス目の値が 1 の場合に実行されるコマンド
- `TuringProgram::Dict{Symbol, TuringInstruction}`: チューリングマシンに実行させるプログラム
    - Fields:
        - `@key::Symbol`: 状態
        - `@val::TuringInstruction`: 状態に対応する命令構造体
- `TuringHead::mutable struct`: ヘッド
    - Fields:
        - `position::Int`: ヘッド位置
    - Functions:
        - `read(self::TuringHead, tape::TuringTape) ::TuringValue`: ヘッドが接するテープのマス目から値を読み取る
        - `write!(tape::TuringTape, self::TuringHead, value::TuringValue)`: ヘッドが接するテープのマス目に値 (0 or 1 or nothing) を書き込む
        - `move!(self::TuringHead, direction::Union{Value{1}, Value{-1}})`: ヘッドを前か後ろに一つ移動する
- `TuringMachine::mutable struct`: チューリングマシン
    - Fields:
        - `head::TuringHead`: ヘッド
        - `tape::TuringTape`: テープ
        - `state::Symbol`: 現在の状態
        - `startState::Symbol`: 初期状態
        - `endState::Symbol`: 終了状態
        - `program::TuringProgram`: 実行するプログラム
    - Functions:
        - `isend(self::TuringMachine) ::Bool`: チューリングマシンが終了状態か判定
        - `setstate!(self::TuringMachine, state::Symbol) ::MaybeTuringInstruction`: チューリングマシンの状態を変更し、次に実行する注文構造体を取得する
            - `MaybeTuringInstruction::Union{TuringInstruction, Nothing}`: TuringInstruction or nothing
        - `getcommand(self::TuringMachine, instruction::TuringInstruction) ::TuringCommand`: ヘッドが指すテープ情報から次に実行するコマンドを取得する
        - `execute!(self::TuringMachine, command::TuingCommand) ::MaybeTuringInstructure`: コマンドを実行（テープへの書き込み・ヘッドの移動を実行）し、次の状態へ遷移する
        - `run!(self::TuringMachine)`: プログラム実行
        - `str(self::TuringMachine) ::AbstractString`: チューリングマシンテープの値を文字列として取得 (nothing は `.` に変換される)
        - `val(self::TuringMachine) ::Int`: チューリングマシンテープの値を10進数の数値として取得

In [1]:
using Match

###############
# definitions #
###############
val(v::Val{T}) where T = T

const TuringValue = Union{Val{0}, Val{1}, Nothing}
const TuringTape = Dict{Int, TuringValue}
const TuringHeadMoveDirection = Union{Val{-1}, Val{1}}

struct TuringCommand
    write::TuringValue
    move::TuringHeadMoveDirection
    next::Symbol
end

struct TuringInstruction
    command::TuringCommand
    command0::TuringCommand
    command1::TuringCommand
end

const MaybeTuringInstruction = Union{TuringInstruction, Nothing}
const TuringProgram = Dict{Symbol, TuringInstruction}

mutable struct TuringHead
    position::Int
end

read(self::TuringHead, tape::TuringTape) ::TuringValue = haskey(tape, self.position) ? tape[self.position] : nothing
write!(tape::TuringTape, self::TuringHead, value::TuringValue) = (tape[self.position] = value)
move!(self::TuringHead, direction::TuringHeadMoveDirection) = (self.position += val(direction))

mutable struct TuringMachine
    head::TuringHead
    tape::TuringTape
    state::Symbol
    startState::Symbol
    endState::Symbol
    program::TuringProgram
end

TuringMachine(program::TuringProgram; tape::TuringTape = TuringTape(), startState::Symbol = :start, endState::Symbol = :end) = TuringMachine(
    TuringHead(1),
    tape,
    startState,
    startState,
    endState,
    program
)

isend(self::TuringMachine) ::Bool = self.state === self.endState

setstate!(self::TuringMachine, state::Symbol) ::MaybeTuringInstruction = begin
    self.state = state
    haskey(self.program, self.state) ? self.program[self.state] : nothing
end

getcommand(self::TuringMachine, instruction::TuringInstruction) ::TuringCommand = @match read(self.head, self.tape) begin
    Val(0) => instruction.command0
    Val(1) => instruction.command1
    _ => instruction.command
end

execute!(self::TuringMachine, command::TuringCommand) ::MaybeTuringInstruction = begin
    write!(self.tape, self.head, command.write)
    move!(self.head, command.move)
    setstate!(self, command.next)
end

run!(self::TuringMachine) = begin
    nextInstruction = setstate!(self, self.startState)

    while nextInstruction !== nothing && !isend(self)
        command = getcommand(self, nextInstruction)
        nextInstruction = execute!(self, command)
    end
end

str(self::TuringMachine) ::AbstractString = map(sort(collect(self.tape); by = pair -> pair[1])) do pair
    isnothing(pair[2]) ? "." : string(val(pair[2]))
end |> join

val(self::TuringMachine) ::Int = filter(sort(collect(self.tape); by = pair -> pair[1])) do pair
    !isnothing(pair[2])
end |> tape -> map(tape) do pair
    string(val(pair[2]))
end |> join |> joinedTape -> parse(Int, joinedTape; base = 2)


"""
@test テープに記述された数値に1を加算するチューリングマシン
"""
# 初期値 `2` を表すテープ: 2進数 `10`
tape = TuringTape(
    1 => Val(1),
    2 => Val(0)
)

# チューリングマシンに与えるプログラム: テープに記述された値に1を加算
program = TuringProgram(
    :q0 => TuringInstruction(
        TuringCommand(nothing, Val(-1), :q1),
        TuringCommand(Val(0), Val(1), :q0),
        TuringCommand(Val(1), Val(1), :q0)
    ),
    :q1 => TuringInstruction(
        TuringCommand(Val(1), Val(-1), :q3),
        TuringCommand(Val(1), Val(-1), :q2),
        TuringCommand(Val(0), Val(-1), :q1)
    ),
    :q2 => TuringInstruction(
        TuringCommand(nothing, Val(1), :q4),
        TuringCommand(Val(0), Val(-1), :q2),
        TuringCommand(Val(1), Val(-1), :q2)
    ),
    :q3 => TuringInstruction(
        TuringCommand(nothing, Val(1), :q4),
        TuringCommand(Val(0), Val(1), :q4),
        TuringCommand(Val(1), Val(1), :q4)
    )
)

machine = TuringMachine(program; tape = tape, startState = :q0, endState = :q4)
run!(machine)

display(machine.tape)
display(str(machine))
display(val(machine))

Dict{Int64, Union{Val{1}, Val{0}, Nothing}} with 4 entries:
  0 => nothing
  2 => Val{1}()
  3 => nothing
  1 => Val{1}()

".11."

3

## 命令型モデルの特徴

- 計算機に命令を下し、計算機はその命令を実行するたびに内部の状態を変化させ、最終的に到達した状態が計算の結果となる
    - チューリングマシンの場合:
        - 命令:
            - `program::TuringProgram`
        - 状態:
            - `TuringMachine.tape::TuringTape`
            - `TuringMachine.state::Symbol`
            - `TuringMachine.head.position::Int`
- 「命令」は処理の一連の流れを記述したものであるため、ある観点から見るととても直感的
    - 例えば、誰かに買い物を頼むとき、店までの道順と買う品物をすべて列挙して頼むことに似ている
    - 買い物を頼まれた側は、依頼された行動を単純に上から順に実行していくだけで良い
- 一方、命令の内容を見ても分かる通り、命令型モデルでは命令の定義が冗長でわかりにくい側面もある
    - これは、チューリングマシンの「状態」が把握しづらく、かつその把握しづらい状態を常に意識して命令を作らなければならないことに起因する
    - チューリングマシンの場合は「テープ」「状態」「ヘッド位置」という3つの状態があり、これらを命令の中で明示的に指し示す方法がないため、プログラムが必要以上に複雑化してしまう